In [ ]:
from matplotlib.dates import relativedelta
from assetAllocation import AssetAllocation
import commonHelper
from dataDownloader import DataDownloader
from db_financialStatement import DB_FinancialStatement
from portfolio import Portfolio
import yfinance as yf
import pandas as pd
import numpy as np

pd.set_option('future.no_silent_downcasting', True)

start_date = '2013-12-01'
end_date = '2025-08-19'

def get_super_quality_rank_table(loaded=True):
    csv_file_name = 'super_quality_rank.csv'
    if loaded:
            result_df = pd.read_csv(csv_file_name)
            result_df = result_df.drop(columns=['Unnamed: 0'], errors='ignore')
            return result_df
    else:
        with DB_FinancialStatement() as fs:
            symbol = fs.get_symbol_list_with_filter(2022)
            symbol = symbol[:10]
            df = fs.get_fs_all(symbol, commonHelper.EDateType.YEAR)
            df = fs.get_fs_data(df)
            df = DB_FinancialStatement.add_year_column(df)

            grouped = df.groupby('Year')
            dict_year = {}
            for y, group in grouped:
                group = group.fillna(0).infer_objects(copy=False)
                group = group[(group['CommonStockIssuance'] == 0) &
                            (group['NetIcomeGrowth'] > 0)]
                dict_year[y] =  [sym for sym in set(group['Symbol'].tolist()) if pd.notna(sym)]
            
            symbol_years = [sym for val in dict_year.values() 
                                for sym in val]
            
            df = fs.get_fs_all(symbol_years, commonHelper.EDateType.QUARTER)
            df = fs.get_fs_data(df)
            df = fs.get_volatility_m(df)
            df = DB_FinancialStatement.add_quarter_column(df)

            grouped = df.groupby('Quarter')
            dict_quarter = {}
            for q, group in grouped:
                year = q.split('-')[0]
                group = group[(group['OperatingCashFlow'] > 0) &
                            (group['Symbol'].isin(dict_year[int(year)-1]))&
                            (group['TotalDebtGrowth'].notna())&
                            (group['TotalDebtGrowth'] != 0)] # 값에 변화가 없을때 제외한다.
                group = group.copy()
                group['Income_to_DebtGrowth'] = group['OperatingIncome'] / group['TotalDebtGrowth']

                if len(group) == 0:
                    continue

                rank_gp_a = group.sort_values(by='GP/A', ascending=False)['Symbol'].tolist()
                rank_income_to_debt = group.sort_values(by='Income_to_DebtGrowth', ascending=False)['Symbol'].tolist() # 영업이익/차입금증감율
                rank_asset_growth = group.sort_values(by='TotalAssetsGrowth')['Symbol'].tolist()
                rank_volatility =group.sort_values(by='Volatility12M')['Symbol'].tolist()

                gp_a_dict = {sym:i for i,sym in enumerate(rank_gp_a)}
                income_to_debt_dict = {sym:i for i,sym in enumerate(rank_income_to_debt)}
                asset_growth_dict = {sym:i for i,sym in enumerate(rank_asset_growth)}
                volatility_dict = {sym:i for i,sym in enumerate(rank_volatility)}

                common_symbol = set(gp_a_dict) & set(income_to_debt_dict) & set (asset_growth_dict) & set(volatility_dict)
                avg_rank = []
                for sym in common_symbol:
                    r1 = gp_a_dict[sym]
                    r2 = income_to_debt_dict[sym]
                    r3 = asset_growth_dict[sym]
                    r4 = volatility_dict[sym]
                    avg = (r1 + r2 + r3 + r4) / 4
                    avg_rank.append((sym, avg))

                avg_rank_sorted = sorted(avg_rank, key = lambda x:x[1])
                final_symbols = [sym for sym, _ in avg_rank_sorted]
                dict_quarter[q] = final_symbols
            
            dict_quarter = DB_FinancialStatement.stuff_df_nan(dict_quarter)
            df = pd.DataFrame(dict_quarter)
            # df.to_csv(csv_file_name)
            return df

Portfolio.super_quality(start_date,end_date)


,Date,Symbol,Result Close,MMA_10,End Balance,Restart Asset,Restart Balance,Cash,Balance,Do Rebalance
0,2024-02-29,"[ADP, AFYA, AIT, ALSN, AMZN, AOS, BBWI, BJ, BR...","[251.13, 20.64, 189.89, 75.33, 176.76, 82.9, 4...","[234.93, 17.11, 157.44, 57.99, 141.28, 73.61, ...",[0],[SPHR],[10000],0,10000,False
1,2024-03-28,"[ADP, AFYA, AIT, ALSN, AMZN, AOS, BBWI, BJ, BR...","[249.74, 18.58, 197.55, 81.16, 180.38, 89.46, ...","[239.0, 17.8, 164.9, 61.37, 147.26, 76.16, 38....",[11334.87],[SPHR],[11334.87],0,11334.87,False
2,2024-04-30,"[ADP, AFYA, AIT, ALSN, AMZN, AOS, BBWI, BJ, BR...","[241.89, 17.23, 183.25, 73.55, 175.0, 82.84, 4...","[241.21, 18.12, 168.74, 63.08, 151.73, 77.16, ...",[8974.59],[SPHR],[8974.59],0,8974.59,False
3,2024-05-31,"[ADP, AFYA, AIT, ALSN, AMZN, AOS, BBWI, BJ, BR...","[244.92, 16.65, 193.0, 75.81, 176.44, 83.64, 5...","[240.98, 18.21, 173.54, 64.8, 156.0, 78.26, 41...",[8429.56],"[BRBR, IDXX, ITRN, OPRA, AFYA, USAC, KORE, ALS...","[648.0, 648.0, 648.0, 648.0, 648.0, 648.0, 648...",0,8429.56,True
4,2024-06-28,"[ADP, AFYA, AIT, ALSN, AMZN, AOS, BBWI, BJ, BR...","[238.69, 17.65, 194.0, 75.9, 193.25, 81.78, 39...","[239.38, 18.47, 177.5, 66.34, 161.53, 79.19, 4...","[636.53, 635.29, 576.62, 652.65, 686.92, 627.6...","[BRBR, IDXX, ITRN, OPRA, AFYA, USAC, KORE, ALS...","[636.53, 635.29, 576.62, 652.65, 686.92, 627.6...",0,7959.29,False
5,2024-07-31,"[ADP, AFYA, AIT, ALSN, AMZN, AOS, BBWI, BJ, BR...","[262.62, 17.95, 218.19, 88.59, 186.98, 85.04, ...","[241.59, 18.68, 183.86, 69.29, 167.51, 81.08, ...","[571.25, 620.84, 619.45, 584.78, 698.6, 606.74...","[BRBR, IDXX, ITRN, OPRA, AFYA, USAC, KORE, ALS...","[571.25, 620.84, 619.45, 584.78, 698.6, 606.74...",0,9351.59,False
6,2024-08-30,"[ADP, AFYA, AIT, ALSN, AMZN, AOS, BBWI, BJ, BR...","[275.91, 16.3, 205.12, 92.75, 178.5, 83.72, 30...","[247.36, 18.7, 189.02, 73.53, 172.05, 82.48, 4...","[623.05, 627.63, 661.11, 692.16, 634.38, 595.3...","[VRSN, NBIX, LOPE, CHE, CTAS, HRB, MTCH, CMCSA...","[521.0, 521.0, 521.0, 521.0, 521.0, 521.0, 521...",0,10433.42,True
7,2024-09-30,"[ADP, AFYA, AIT, ALSN, AMZN, AOS, BBWI, BJ, BR...","[276.73, 17.07, 223.13, 96.07, 186.33, 89.83, ...","[252.04, 18.36, 195.33, 77.78, 176.08, 83.93, ...","[538.17, 472.45, 509.65, 534.15, 532.91, 522.9...","[VRSN, NBIX, LOPE, CHE, CTAS, HRB, MTCH, CMCSA...","[538.17, 472.45, 509.65, 534.15, 532.91, 522.9...",0,10503.82,False
8,2024-10-31,"[ADP, AFYA, AIT, ALSN, AMZN, AOS, BBWI, BJ, BR...","[289.24, 16.76, 231.59, 106.86, 186.4, 75.1, 2...","[257.67, 17.84, 201.22, 82.66, 179.52, 83.19, ...","[501.0, 493.16, 492.62, 480.17, 532.73, 491.54...","[VRSN, NBIX, LOPE, CHE, CTAS, HRB, MTCH, CMCSA...","[501.0, 493.16, 492.62, 480.17, 532.73, 491.54...",0,10235.83,False
9,2024-11-29,"[ADP, AFYA, AIT, ALSN, AMZN, AOS, BBWI, BJ, BR...","[306.93, 16.23, 274.72, 118.5, 207.89, 74.49, ...","[263.78, 17.51, 211.04, 88.45, 184.79, 82.88, ...","[530.29, 519.73, 591.35, 508.75, 584.45, 487.8...","[DPZ, MO, MSM, USFD, CTAS, PEP, YUM, STZ, PHI,...","[540.0, 540.0, 540.0, 540.0, 540.0, 540.0, 540...",0,10801.63,True
